In [1]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import pandas as pd
import matplotlib.patches as patches
from matplotlib.path import Path
from matplotlib.patches import PathPatch
%matplotlib qt

In [3]:
path='../Sphere'

In [176]:
#Load preprocessed data and targets
data = pd.read_csv(path+'/data/train/00001/columns_1000ms.csv')
targets = pd.read_csv(path+'/data/train/00001/targets.csv')

In [177]:
# Make a dataframe with just the 2d video bounding box values
boundings = data[['video_hallway_bb_2d_br_y_mean', 
             'video_hallway_bb_2d_tl_y_mean',
             'video_hallway_bb_2d_br_x_mean',
             'video_hallway_bb_2d_tl_x_mean',
             'video_kitchen_bb_2d_br_y_mean', 
             'video_kitchen_bb_2d_tl_y_mean',
             'video_kitchen_bb_2d_br_x_mean',
             'video_kitchen_bb_2d_tl_x_mean',
             'video_living_room_bb_2d_br_y_mean', 
             'video_living_room_bb_2d_tl_y_mean',
             'video_living_room_bb_2d_br_x_mean',
             'video_living_room_bb_2d_tl_x_mean']]

accelerations = data[['acceleration_x_mean',
                     'acceleration_y_mean',
                     'acceleration_x_mean']]


In [259]:
# Create list of labels for each 1 second interval using argmax on the targets from the annotators
labels = []
for i in range(len(targets)):
    
    # Determine the index of the label which has the largest target value in the list of actions
    label_ind = np.argmax(targets.iloc[i][2:])
    
    if label_ind == -1:
        label = 'unknown'
        
    # If not unknown, assign to label the name of the column which has the largest target value
    else:
        label = targets.columns[label_ind + 2]
        
    labels.append(label)
    
column_names = targets.columns[2:]

In [257]:
def animate():
    
    #################################
    #### set up axes and patches ####
    #################################

    fig = plt.figure(figsize=(8,8),tight_layout=True)
    gs = fig.add_gridspec(3, 3, height_ratios=[5,5,1], width_ratios=[6,1,2])
    
    
    camera = fig.add_subplot(gs[0, :2])
    camera.set_xlim([0, 320])
    camera.set_ylim([0, 240])
    camera.set_title('Camera: click to pause')
    camera_name = camera.text(0.5,0.1,'unknown', ha="center",va="center", color='white',
                     transform=camera.transAxes, fontsize="large")
    im = camera.imshow(np.ones((320,240)), extent=[0, 320, 0, 240])
    camera.invert_xaxis()
 
    
    l_im = plt.imread(path+"/data/supplementary/living_room.png")
    h_im = plt.imread(path+"/data/supplementary/hallway.png")
    k_im = plt.imread(path+"/data/supplementary/kitchen.png")

 
    floor = fig.add_subplot(gs[1, :2])
    floor.set_title('Floorplan')
    floor.set_xlim([0, 320])
    floor.set_ylim([0, 240])
    f_im = plt.imread(path+"/data/supplementary/floors.png")
    floor.imshow(f_im, extent=[0, 320, 0, 240])
    person = patches.Ellipse(xy=(-100,-100), width=20, height=20, fc='y', ec='k',zorder=100)
    floor.add_patch(person)

    
    time = fig.add_subplot(gs[2,:])
    t2 = targets.fillna(0).drop(columns=['start','end']).transpose()
    time.set_ylim([0, 20])
    time.set_xlim([0, boundings.shape[0]])
    time.imshow(t2, aspect='auto',interpolation='none', alpha=1,cmap='GnBu')
    time.set_title("Time: click to change time") 
    time.invert_yaxis()
    
    
    pose = fig.add_subplot(gs[:2, 2])
    pose.set_title('Pose')
    pose_text = pose.text(0.5,0.97,'', ha="center",va="center", color='black', fontsize="medium",
                         transform=pose.transAxes, )
    pose.barh(list(t2.index.values), np.zeros(20), color = 'salmon')
    pose.invert_yaxis()
    
    
    axes = (camera, floor, pose, time)
    
    for ax in axes:
        if ax == pose:
            ax.axes.xaxis.set_ticks([])

        else:      
            ax.axes.xaxis.set_ticks([])
            ax.axes.yaxis.set_ticks([])
       
    
    line, = camera.plot([], [], lw=3)
    tracker = time.axvline(0, 0, 1)
    clock = time.text(0.01, 0.5,'',color='black',ha="left",va="center",
                     transform=time.transAxes, fontsize="medium")
    clock_tot = time.text(0.99, 0.5,f'{boundings.shape[0]}',color='black',ha="right",va="center",
                     transform=time.transAxes, fontsize="medium")
    
    
    paused = [ False ]
    last_index = [ -1 ]
    t_index = [ 0 ]
    
    ######################
    #### move patches ####
    ######################

    def draw(index):

        if not paused[0]:
            t_index[0] = t_index[0] + (index - last_index[0])
            t_index[0] = t_index[0] % boundings.shape[0]

        last_index[0] = index

        # set patches ONLY WHEN camera data available
        bb_indexes = np.where(np.isnan(boundings.iloc[t_index[0]])==False)[0]
        if bb_indexes.size > 0:

            bbs = boundings.iloc[t_index[0]][bb_indexes]

            if bb_indexes[0] == 0:
                im.set_array(h_im)
                camera_name.set_text('Hallway')
                person.center = 160, 150

            elif bb_indexes[0] == 4:
                im.set_array(k_im)
                camera_name.set_text('Kitchen')
                person.center = 135, 55

            elif bb_indexes[0] == 8:
                im.set_array(l_im)
                camera_name.set_text('Living Room')
                person.center = 115, 180

            x_vals = [bbs[2],bbs[2],bbs[3],bbs[3],bbs[2]]
            y_vals = [240-bbs[1],240-bbs[0],240-bbs[0],240-bbs[1],240-bbs[1]]
            line.set_data(x_vals, y_vals)
            line.set_color('yellow')
            
        # set patches ONLY WHEN camera data NOT available    
        else:
            line.set_data([],[])
            im.set_array(np.ones((320,240)))
            camera_name.set_text('unknown')
            person.center = -100,-100
        

        # set patches ALL THE TIME REGARDLESS OF camera data availability  
        heights = targets.iloc[t_index[0]][2:]
        pose.barh(list(t2.index.values), heights, color = 'salmon')
        for bar in pose.containers:
            bar.remove()
        for j, b in enumerate(pose.containers):
            b[0].set_height(heights[j])       
        
        pose_text.set_text(labels[t_index[0]])     
        tracker.set_xdata([t_index[0], t_index[0]])
        clock.set_text(f'{t_index[0]}')
        
        super_tuple = (person, pose, clock, camera_name, im, tracker, line,) #line, # tuple of patches tuple(l_patches)+(trail, clock, tracker, intensity, location, angle, score) + agent_patches
        return super_tuple
    
    
    #################################
    #### set up animation params ####
    #################################
    

    def init():
        result = draw(0)
        return result

    
    def onclick(event):
        if event.button == 1:
            # pause if the user clicks on the main figure
            if event.inaxes is camera:
                paused[0] = not paused[0]
            # edit time directly if the user clicks on the graph over time
            elif event.inaxes is time:
                t_index[0] = (int) (event.xdata)            

                
    def anim(index):
        return draw(index)


    ani = FuncAnimation(fig, anim, init_func=init, frames=boundings.shape[0], interval=100, blit=True, repeat=False)

    fig.canvas.mpl_connect('button_press_event', onclick)
    plt.show()
    return ani

In [258]:
############################################################
############################################################
ani = animate()
############################################################
############################################################

In [117]:
# HTML(ani.to_html5_video())

## Arrows included

In [243]:
def animate():
    
    #################################
    #### set up axes and patches ####
    #################################

    fig = plt.figure(figsize=(10,8),tight_layout=True)
    gs = fig.add_gridspec(5, 3, height_ratios=[4,3,3,4,2],width_ratios=[6,4,3])
    
    #### camera ####
    camera = fig.add_subplot(gs[:2, :1])
    camera.set_xlim([0, 320])
    camera.set_ylim([0, 240])
    camera.set_title('Camera: click to pause')
    camera_name = camera.text(0.5,0.1,'unknown', ha="center",va="center", color='white',
                     transform=camera.transAxes, fontsize="large")
    im = camera.imshow(np.ones((320,240)), extent=[0, 320, 0, 240])
    camera.invert_xaxis()
    
    l_im = plt.imread(path+"/data/supplementary/living_room.png")
    h_im = plt.imread(path+"/data/supplementary/hallway.png")
    k_im = plt.imread(path+"/data/supplementary/kitchen.png")
    
    line, = camera.plot([], [], lw=3)

    
    #### floorplan ####
    floor = fig.add_subplot(gs[2:4, :1])
    floor.set_title('Floorplan')
    floor.set_xlim([0, 320])
    floor.set_ylim([0, 240])
    f_im = plt.imread(path+"/data/supplementary/floors.png")
    floor.imshow(f_im, extent=[0, 320, 0, 240])
    
    person = patches.Ellipse(xy=(-100,-100), width=20, height=20, fc='y', ec='k',zorder=100)
    floor.add_patch(person)
    
    
    #### accelerometer ####
    acc = fig.add_subplot(gs[1:3,1])
    acc.set_title('Acceleration')
    acc.set_xlim([-1.1,1.1])
    acc.set_ylim([-1.5,1.1])
    
    # arrow lines
    linex, = acc.plot([], [], lw=4, label='x acceleration')
    liney, = acc.plot([], [], lw=4, label='y acceleration')
    linez, = acc.plot([], [], lw=4, label='z acceleration')
    
    # arrow heads
    x_tri = patches.Polygon([(0,0),(0,0),(0,0)], color='#1f77b4')
    y_tri = patches.Polygon([(0,0),(0,0),(0,0)], color = '#ff7f0e')
    z_tri = patches.Polygon([(0,0),(0,0),(0,0)], color = '#2ca02c')
    
    acc.add_patch(x_tri)
    acc.add_patch(y_tri)
    acc.add_patch(z_tri)
    
    acc.legend(loc='lower left')
    
    
    #### pose predictions ####
    pose = fig.add_subplot(gs[:4, 2])
    pose.set_title('Pose')
    pose_text = pose.text(0.5,0.975,'', ha="center",va="center", color='black', fontsize="medium",
                          transform=pose.transAxes)
    pose.barh(column_names, np.zeros(20), color = 'salmon')
    pose.invert_yaxis()
    
    
    #### time bar ####
    time = fig.add_subplot(gs[4,:])
    t2 = targets.fillna(0).drop(columns=['start','end']).transpose()
    time.set_ylim([0, 20])
    time.set_xlim([0, boundings.shape[0]])
    time.imshow(t2, aspect='auto',interpolation='none', alpha=1,cmap='GnBu')
    time.set_title("Time: click to change time") 
    time.invert_yaxis()
    
    tracker = time.axvline(0, 0, 1)
    clock = time.text(0.01, 0.5,'',color='black',ha="left",va="center",
                     transform=time.transAxes, fontsize="medium")
    clock_tot = time.text(0.99, 0.5,f'{boundings.shape[0]}',color='black',ha="right",va="center",
                     transform=time.transAxes, fontsize="medium")
    
    
    #### set up axes ####
    axes = (camera, floor, pose, time, acc)
    
    for ax in axes:
        if ax == pose:
            ax.axes.xaxis.set_ticks([])
        else:      
            ax.axes.xaxis.set_ticks([])
            ax.axes.yaxis.set_ticks([])
            
            
    #### set up pause function ####    
    paused = [ False ]
    last_index = [ -1 ]
    t_index = [ 0 ]
    
    
    
    ######################
    #### move patches ####
    ######################

    def draw(index):

        if not paused[0]:
            t_index[0] = t_index[0] + (index - last_index[0])
            t_index[0] = t_index[0] % boundings.shape[0]

        last_index[0] = index
        
        
        #### CAMERA PATCHES ONLY (+ person loc) ####
        
        # set camera patches ONLY WHEN camera data available
        bb_indexes = np.where(np.isnan(boundings.iloc[t_index[0]])==False)[0]
        if bb_indexes.size > 0:

            bbs = boundings.iloc[t_index[0]][bb_indexes]

            if bb_indexes[0] == 0:
                im.set_array(h_im)
                camera_name.set_text('Hallway')
                person.center = 160, 150

            elif bb_indexes[0] == 4:
                im.set_array(k_im)
                camera_name.set_text('Kitchen')
                person.center = 135, 55

            elif bb_indexes[0] == 8:
                im.set_array(l_im)
                camera_name.set_text('Living Room')
                person.center = 115, 180

            x_vals = [bbs[2],bbs[2],bbs[3],bbs[3],bbs[2]]
            y_vals = [240-bbs[1],240-bbs[0],240-bbs[0],240-bbs[1],240-bbs[1]]
            line.set_data(x_vals, y_vals)
            line.set_color('yellow')
        
        # set camera patches ONLY WHEN camera data NOT available
        else:
            line.set_data([],[])
            im.set_array(np.ones((320,240)))
            camera_name.set_text('unknown')
            person.center = -100,-100
            
            
        #### ALL OTHER PATCHES #### 
        
        #### pose patches ####
        heights = targets.iloc[t_index[0]][2:]
        pose.barh(column_names, heights, color = 'salmon')
        for bar in pose.containers:
            bar.remove()
        for j, b in enumerate(pose.containers):
            b[0].set_height(heights[j])
        pose_text.set_text(labels[t_index[0]])


        #### accelerometer patches ####    
        # Acceleration arrows update
        xacc = accelerations.iloc[t_index[0]][0]
        yacc = accelerations.iloc[t_index[0]][1]
        zacc = accelerations.iloc[t_index[0]][2]

        # Set the lines
        linex.set_data([0,0.866*xacc],[0,-0.5*xacc])
        liney.set_data([0,0.866*yacc],[0,0.5*yacc])
        linez.set_data([0,0],[0,zacc])

        # Set the triangles at the end of the lines
        tri_param = 0.05
        tri_param2 = 0.04

        if xacc > 0:
            x_tri.set_xy([(0.866*xacc-tri_param2,-0.5*xacc-tri_param2),
                      (0.866*xacc+tri_param2,-0.5*xacc-tri_param2),
                      (0.866*xacc+tri_param2,-0.5*xacc+tri_param2)])
        else:
            x_tri.set_xy([(0.866*xacc-tri_param2,-0.5*xacc-tri_param2),
                      (0.866*xacc-tri_param2,-0.5*xacc+tri_param2),
                      (0.866*xacc+tri_param2,-0.5*xacc+tri_param2)])

        if yacc > 0:
            y_tri.set_xy([(0.866*yacc-tri_param2,0.5*yacc+tri_param2),
                      (0.866*yacc+tri_param2,0.5*yacc+tri_param2),
                      (0.866*yacc+tri_param2,0.5*yacc-tri_param2)])
        else:
            y_tri.set_xy([(0.866*yacc+tri_param2,0.5*yacc-tri_param2),
                      (0.866*yacc-tri_param2,0.5*yacc-tri_param2),
                      (0.866*yacc-tri_param2,0.5*yacc+tri_param2)])

        if zacc > 0:
            z_tri.set_xy([(-tri_param,zacc),
                      (0,zacc+tri_param),
                      (+tri_param,zacc)])
        else:
            z_tri.set_xy([(tri_param,zacc),
                      (0,zacc-tri_param),
                      (-tri_param,zacc)])
        
        #### clock patches ####
        tracker.set_xdata([t_index[0], t_index[0]])
        clock.set_text(f'{t_index[0]}')
        
        #### add all patches into a tuple ####
        super_tuple = (person, pose, pose_text, linex, liney, linez, x_tri, y_tri, z_tri, clock, camera_name, im, tracker, line,) #line, # tuple of patches tuple(l_patches)+(trail, clock, tracker, intensity, location, angle, score) + agent_patches
        return super_tuple
    
    
    #################################
    #### set up animation params ####
    #################################
    

    def init():
        result = draw(0)
        return result

    
    def onclick(event):
        if event.button == 1:
            # pause if the user clicks on the main figure
            if event.inaxes is camera:
                paused[0] = not paused[0]
            # edit time directly if the user clicks on the graph over time
            elif event.inaxes is time:
                t_index[0] = (int) (event.xdata)            

                
    def anim(index):
        return draw(index)


    ani = FuncAnimation(fig, anim, init_func=init, frames=boundings.shape[0], interval=100, blit=True, repeat=False)

    fig.canvas.mpl_connect('button_press_event', onclick)
    plt.show()    
    return ani

In [244]:
############################################################
############################################################
ani = animate()
############################################################
############################################################